In [10]:
%load_ext autoreload
%autoreload 2
import torch
import eq
import wandb
from tqdm.notebook import trange
import numpy as np
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
device = "cuda"
catalog = eq.catalogs.ANSS_MultiCatalog(mag_completeness=4.5)

Loading existing catalog from /home/zekai/repos/recast/data/ANSS_MultiCatalog.


In [12]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        try:
            dl_train = catalog.train.get_dataloader(batch_size=config.batch_size)
            dl_val = catalog.val.get_dataloader(batch_size=1)
            dl_test = catalog.test.get_dataloader(batch_size=1)

            model = eq.models.RecurrentTPP(context_size=config.context_size,
                                           num_components=config.num_components,
                                           rnn_type=config.rnn_type,
                                           dropout_proba=config.dropout_proba,
                                           learning_rate=config.lr)
            model = model.to(device)

            epochs = 200
            avg_train_loss_list = []
            avg_val_loss_list = []

            optimizer = torch.optim.AdamW(model.parameters(), 
                                          lr=config.lr, 
                                          betas=config.betas, 
                                          weight_decay=config.weight_decay)
            
            best_model_path = "temp_best_model"
            best_val_loss = float('inf')

            for epoch in trange(epochs):
                running_training_loss = []
                model.train()
                for i, data in enumerate(dl_train):
                    data = data.to(device)
                    optimizer.zero_grad()
                    nll = model.nll_loss(data).mean()
                    nll.backward()
                    optimizer.step()
                    running_training_loss.append(nll.item())
                
                model.eval()
                with torch.no_grad():
                    running_val_loss = []
                    for i, data in enumerate(dl_val):
                        data = data.to(device)
                        nll = model.nll_loss(data).mean()
                        running_val_loss.append(nll.item())

                avg_val_loss = np.mean(running_val_loss)

                avg_train_loss_list.append(np.mean(running_training_loss))
                avg_val_loss_list.append(avg_val_loss)

                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    torch.save(model.state_dict(), best_model_path)

            best_model = eq.models.RecurrentTPP(context_size=config.context_size,
                                                num_components=config.num_components,
                                                rnn_type=config.rnn_type,
                                                dropout_proba=config.dropout_proba,
                                                learning_rate=config.lr)
            best_model.load_state_dict(torch.load(best_model_path))
            best_model = best_model.to(device)
            best_model.eval()
            with torch.no_grad():
                running_test_loss = []
                for i, data in enumerate(dl_test):
                    data = data.to(device)
                    nll = best_model.nll_loss(data).mean()
                    running_test_loss.append(nll.item())
            avg_test_loss = np.mean(running_test_loss)

        except Exception as e:
            print(e)
            avg_test_loss = float("nan")

        wandb.log({"avg_test_loss": avg_test_loss})


In [13]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "sweep.ipynb"
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [14]:
sweep_config = {'method': "random"}
metric = {"name": "avg_test_loss",
          "goal": "minimize"}
sweep_config["metric"] = metric

parameter_dict = {"context_size": {'values': [8, 16, 32, 64, 128]},
                  "num_components": {"values": [8, 16, 32, 64, 128]},
                  "rnn_type": {"values": ["RNN", "GRU", "LSTM"]},
                  "dropout_proba": {"values": [0, 0.1, 0.2, 0.3, 0.4, 0.5]},
                  "lr": {"values": [1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2]},
                  "betas": {"value": (0.9, 0.999)},
                  "weight_decay": {"values": [0, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2]},
                  "batch_size": {"values": [8, 16, 32, 64, 128]}}

sweep_config["parameters"] = parameter_dict

In [15]:
sweep_id = wandb.sweep(sweep_config, project="RecurrentTPP on ANSS Nov 5")

Create sweep with ID: 27ca0onv
Sweep URL: https://wandb.ai/zekai-wang/RecurrentTPP%20on%20ANSS%20Nov%205/sweeps/27ca0onv


In [16]:
wandb.agent(sweep_id, function=train, count=100)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


<IPython.core.display.HTML object>


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: Agent Starting Run: tdq51qjk with config:
wandb: 	batch_size: 128
wandb: 	betas: [0.9, 0.999]
wandb: 	context_size: 8
wandb: 	dropout_proba: 0
wandb: 	lr: 0.01
wandb: 	num_components: 128
wandb: 	rnn_type: GRU
wandb: 	weight_decay: 0.05


...

wandb: Agent Starting Run: e8dapfta with config:
wandb: 	batch_size: 128
wandb: 	betas: [0.9, 0.999]
wandb: 	context_size: 32
wandb: 	dropout_proba: 0.2
wandb: 	lr: 0.002
wandb: 	num_components: 32
wandb: 	rnn_type: RNN
wandb: 	weight_decay: 0.001


  0%|          | 0/200 [00:00<?, ?it/s]

avg_test_loss,▁
avg_test_loss,0.17972


wandb: Agent Starting Run: e2f7bwjk with config:
wandb: 	batch_size: 32
wandb: 	betas: [0.9, 0.999]
wandb: 	context_size: 8
wandb: 	dropout_proba: 0.3
wandb: 	lr: 0.001
wandb: 	num_components: 32
wandb: 	rnn_type: GRU
wandb: 	weight_decay: 0.01


  0%|          | 0/200 [00:00<?, ?it/s]